In [ ]:
from django.http.response import Http404, HttpResponseRedirect
from django.urls import reverse
from django.shortcuts import render, get_list_or_404, get_object_or_404
from django.views.decorators.clickjacking import xframe_options_sameorigin
from django.http import HttpResponse
from asgiref.sync import sync_to_async
from django.forms.models import model_to_dict

import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from account.models import AUser
from tool.models import UserServerRole, ServerRole, Server, CategoryInServer, ChannelOfChat, ChannelOfVoice
# from account.models import BUser
from django.contrib.auth.models import User as BUser
from django.utils import timezone

import sys
import datetime
import json
import random
import string
import hashlib
import base64
from django.db import transaction
# @sync_to_async
# def r():
#     pass

# await r()

In [ ]:
ChannelOfChat.__name__

In [ ]:
GroupMessage.objects.get(pk=1).sender

In [ ]:
from django.db.models import Field

for a_user in AUser.objects.all():
    try:
        # Get all field names of BUser model
        b_user_fields = set(f.name for f in BUser._meta.get_fields() if isinstance(f, Field))
        
        # Create a dictionary of field names and values from AUser
        # Only include fields that exist in BUser
        user_data = {
            field: getattr(a_user, field)
            for field in a_user.__dict__
            if field in b_user_fields and field not in ['id','username']
        }
        user_data['username'] = a_user.username + '_1'
        # Create the BUser object with the filtered data
        BUser.objects.create(**user_data)
    except ValueError:
        pass

In [ ]:
with transaction.atomic():
    from project.snowflake import getToolServerSnowflakeID
    for c in CategoryInServer.objects.all():
        new_uc = getToolServerSnowflakeID()
        CategoryInServer.objects.filter(pk=c.pk).update(urlCode=new_uc)
        ChannelOfChat.objects.filter(category=c).update(category_id=new_uc)
        ChannelOfVoice.objects.filter(category=c).update(category_id=new_uc)
        

In [ ]:
def hash_and_compare(cracked_hash):
    cracked_chain = cracked_hash.split('$')
    # cracked_chain_digest = cracked_chain[0]
    cracked_chain_rounds = int(cracked_chain[1])
    cracked_chain_salt = cracked_chain[2].encode('utf-8')
    cracked_chain_hash_data = cracked_chain[3].split(':')
    cracked_chain_hash = base64.b64decode(cracked_chain_hash_data[0].encode('utf-8'))
    cracked_chain_data = cracked_chain_hash_data[1]

    hash_value = hashlib.pbkdf2_hmac('sha256', cracked_chain_data.encode('utf-8'), cracked_chain_salt, cracked_chain_rounds)
    hash_str = base64.b64encode(hash_value).decode('utf-8')
    salt_str = base64.b64encode(cracked_chain_salt).decode('utf-8')

    print(f'Computed: Hash: {hash_str} Salt: {salt_str}\nCracked: Hash: {cracked_chain_hash_data[0]} Salt: {cracked_chain[2]}\n')

hash_and_compare('pbkdf2_sha256$10000$005OtPxTXhPq$K/2GplWPJsBVj+qbgdKW8YEteQyUkIiquT5MaOhPo4Y=:harry')
hash_and_compare('pbkdf2_sha256$10000$00Qhibr5Mbeg$l9grYueDrl3qN3NA7e9j5PodgV1XkGTz0Z6ajhF99AY=:radio')
hash_and_compare('pbkdf2_sha256$10000$00h7h0g1ZKE1$YEobSm/y+cFg/VXhU4gGYJ6eOkZ68jhJ5axDu68Dack=:momo')
hash_and_compare('pbkdf2_sha256$10000$01JMkfGk1RXh$vD+GGZshw5kExtZOpl5+Lht3xECULdbNVOesoTicxto=:fred')
hash_and_compare('pbkdf2_sha256$10000$01vkw1viCg4J$2hjlbq10Jh/Su3yqjKfYCnCSt1WlKcKJtsqDET618M0=:get')
hash_and_compare('pbkdf2_sha256$10000$01wayF5JLVSZ$2/9COWqb6SZG/raqabtU8fNBzkrt2puN7SaKw0U7jBs=:987456321')


In [ ]:

hashlib.pbkdf2_hmac('sha256', '...'.encode('utf-8'), '...'.encode('utf-8'), 123456).hex()

code

In [ ]:
(AUser.objects.get()).__class__.__name__